In [36]:
import time 

# Importing the libraries
import configparser
import tweepy
import pandas as pd

import uuid

from datetime import datetime
data_ref=datetime.today().strftime("%Y-%m-%d")

# Read the config file
config = configparser.ConfigParser()
config.read('../config.ini_local')

# Read the values
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']


# Authenticate
auth = tweepy.OAuth1UserHandler(
   api_key, api_key_secret, access_token, access_token_secret
)

api = tweepy.API(auth)

In [1]:
# Import libraries
import glob
import pandas as pd

In [2]:
path = "../data/influencer_tweets/"

csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df   = pd.concat(df_list, ignore_index=True)

In [3]:
df.shape

(56100, 6)

In [4]:
df.head()

,id,userID,created_at,full_text,favorite_count,retweet_count
0,1644820236749635591,BRKsEDU,2023-04-08 21:51:18+00:00,@MarioThael Muito obrigado!!! Quem editou foi ...,4,1
1,1644800548812587008,BRKsEDU,2023-04-08 20:33:04+00:00,@V_JuniorR Valeu,1,0
2,1644800516403126274,BRKsEDU,2023-04-08 20:32:56+00:00,@vinossauro Sim e sim 😀,3,0
3,1644800453090091011,BRKsEDU,2023-04-08 20:32:41+00:00,@vsouzaduart Muito obrigado!!!,1,0
4,1644789241619120133,BRKsEDU,2023-04-08 19:48:08+00:00,Acabei de publicar no canal minha crítica ao f...,751,21


In [17]:
df[df.favorite_count > 5000].userID.value_counts()

Cellbit      9364
BRKsEDU       200
Loiranoob     100
Name: userID, dtype: int64

In [43]:
df[df.favorite_count > 5000].shape

(9664, 6)

In [39]:
# Iterar sobre os tweets e imprimir o texto de cada um
def get_text_retweet(api, post_id):

    post_id = 1646187917507801088

    # Coletar a publicação
    post = api.get_status(post_id)

    # Fazer uma busca por tweets que sejam respostas à publicação
    replies = tweepy.Cursor(api.search_tweets, q="to:{}".format(post.author.screen_name),
                            since_id=post.id, tweet_mode="extended").items()

    lista_replies = []
    lista_tweet = []
    uui_id = str(uuid.uuid4())
    for tweet in replies:
        if hasattr(tweet, "in_reply_to_status_id_str"):
            if tweet.in_reply_to_status_id_str == post.id_str:
                # print(tweet.full_text)
                lista_replies.append(tweet)
                lista_tweet.append({
                    "full_text": tweet.full_text,
                    "created_at": tweet.created_at,
                    "id":tweet.id_str 
                })
    d = pd.DataFrame(lista_tweet).to_csv(f'../data/tweet_iteracoes/{post_id}_{data_ref}_{uui_id}.csv', index=False)

In [38]:
#get_text_retweet(api, 1646187917507801088)

In [50]:
for data_loop in df[df.favorite_count > 5000].id.to_list():
    get_text_retweet(api, data_loop)
    time.sleep(4)

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded